In [3]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 26 16:42:46 2025

@author: 86435
"""
import os
import pandas as pd
import numpy as np


data = pd.read_csv("D:/project_h2/data/heating_techs/Biomass_Heating_Technologies.csv")


# Remove ' heat' from bus names
data['bus'] = data['bus'].str.replace(' heat', '', regex=False)

# Prepare data for removal file
data.rename(columns={'decommissioning_year': 'year_removed'}, inplace=True)
removal = data[['year_removed', 'carrier', 'p_nom', 'bus']]


# Group and process the data for each removal year
years_res = sorted(removal.year_removed.unique())
remove = pd.DataFrame(columns=removal.columns)

for yr in years_res:
    dat = removal[removal.year_removed == yr]
    df = dat.groupby(['year_removed', 'bus', 'carrier']).sum().sum(level=['year_removed', 'bus', 'carrier']).unstack('year_removed').fillna(0).reset_index()
    df['year'] = df.columns[2][1]
    df.columns = ['bus', 'carrier', 'p_nom', 'year_removed']
    remove = pd.concat([df, remove], ignore_index=False)

# Final formatting and save
remove.index = remove.bus + ' ' + remove.carrier
output_path = "D:/project_h2/data/heating_techs/biomass_heat_basic_removal.csv"
remove.to_csv(output_path)




C:\Users\86435\AppData\Local\Temp\ipykernel_14140\4231341126.py:29: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df = dat.groupby(['year_removed', 'bus', 'carrier']).sum().sum(level=['year_removed', 'bus', 'carrier']).unstack('year_removed').fillna(0).reset_index()
C:\Users\86435\AppData\Local\Temp\ipykernel_14140\4231341126.py:29: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df = dat.groupby(['year_removed', 'bus', 'carrier']).sum().sum(level=['year_removed', 'bus', 'carrier']).unstack('year_removed').fillna(0).reset_index()
C:\Users\86435\AppData\Local\Temp\ipykernel_14140\4231341126.py:29: FutureWarning: Using the level keyword in DataFrame and Series aggregation

1. Aggregate the total removal capacity per bus and carrier:

In [4]:
correct = remove.groupby(['carrier', 'bus']).agg({'p_nom': ['sum']})
correct = correct.stack().reset_index()
correct = correct[['carrier', 'p_nom', 'bus']]
correct.index = correct.bus + ' ' + correct.carrier


In [5]:
correct

,carrier,p_nom,bus
DE0 2 biomass,biomass,6386.18,DE0 2
DE0 9 biomass,biomass,3999.30,DE0 9
DE0 A biomass,biomass,7686.63,DE0 A
DE0 E biomass,biomass,3805.48,DE0 E


2. Ensure a biomass generator exists for each bus-carrier combo, and correct its capacity:


In [ ]:
n.generators.loc[i, 'p_nom'] = correct.loc[i, 'p_nom']


4. Remove any biomass generator that is not in the removal plan:

In [ ]:
car = 'biomass'
mo = set(n.generators.loc[n.generators.carrier == car, 'bus'])
ac = set(remove.loc[remove.carrier == car, 'bus'])

for item in list(mo.difference(ac)):
    n.remove('Generator', f'{item} {car}')


In [13]:
df=remove
# Biomass burner properties
biomass_efficiency = 0.85
biomass_vom = 50    # Variable operation cost [€/MWh]
biomass_fom = 0.03  # Fixed O&M cost share
biomass_capex = 1200  # €/kW
lifetime = 20
discount_rate = 0.07  # assumed

# Calculate annuity
annuity = discount_rate / (1 - 1 / (1 + discount_rate)**lifetime)

for i in df.index:
    row = df.loc[i]
    bus_ac = row["bus"]  # e.g., "DE09"
    bus_heat = f"{bus_ac} heat"
    tech = "biomass_burner"

    link_name = f"{bus_ac} {tech}"

In [ ]:
bus_ac, row

(DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 DE0 E biomass    DE0 E
 Name: bus, dtype: object,
                  bus  carrier   p_nom year_removed
 DE0 E biomass  DE0 E  biomass  271.82         2045
 DE0 E biomass  DE0 E  biomass  271.82         2043
 DE0 E biomass  DE0 E  biomass  271.82         2041
 DE0 E biomass  DE0 E  biomass  271.82         2039
 DE0 E biomass  DE0 E  biomass  271.82         2037
 DE0 E biomass  DE0 E  biomass  271.82         2035
 DE0 E biomass  DE0 E  biomass  271.82         2033
 DE0 E biomass  DE0 E  biomass  271.82         2032
 DE0 E biomass  DE0 E  biomass  271.82         2030
 DE0 E biomass  DE0 E  biomass  271.82         2028
 DE0 E biomass  DE0 E  biomass  271.82         2026
 DE0 E biomas

In [18]:
bus_heat

'DE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nDE0 E biomass    DE0 E\nName: bus, dtype: object heat'